In [11]:
import pretty_midi
import torch


def melody_to_numpy(fpath="ashover12.mid", unit_time=0.125):
    # 首先取出midi中的音乐信息，将第一个音轨提出来，然后取所有音符元素。
    music = pretty_midi.PrettyMIDI(fpath)
    notes = music.instruments[0].notes

    # 纪录一个时间戳 t和保存向量的列表 roll。
    t = 0.
    roll = list()
    # print(notes[0], notes[-1])

    # 在 Notes里遍历，找出所有音符的音高和长度，同时不放过休止符。
    for note in notes:
        # print(t, note)

        # 两个相邻的音符不是无缝连接，说明休止符存在。计算其相对于最小分辨率 unit_time的相对时长 T，建立一个(T, 130)的矩阵，将第129维置1.
        elapsed_time = note.start - t
        if elapsed_time > 0.:
            steps = torch.zeros((int(round(elapsed_time / unit_time)), 130))
            steps[range(int(round(elapsed_time / unit_time))), 129] += 1.
            roll.append(steps)

        # 如果是无缝连接，那么检查当前音符：
        n_units = int(round((note.end - note.start) / unit_time))
        steps = torch.zeros((n_units, 130))
        steps[0, note.pitch] += 1
        steps[range(1, n_units), 128] += 1

        # 其中除第一列记录pitch外，其他列都记录sustain的128.最后合成为一个矩阵：
        roll.append(steps)
        t = note.end
    return torch.cat(roll, 0)

In [12]:
import numpy as np

def numpy_to_midi(sample_roll, output='sample1.mid'):
    music = pretty_midi.PrettyMIDI()
    piano_program = pretty_midi.instrument_name_to_program(
        'Acoustic Grand Piano')
    piano = pretty_midi.Instrument(program=piano_program)
    t = 0
    for i in sample_roll:
        if 'torch' in str(type(i)):
            pitch = int(i.max(0)[1])
        else:
            pitch = int(np.argmax(i))
        if pitch < 128:
            note = pretty_midi.Note(
                velocity=100, pitch=pitch, start=t, end=t + 1 / 8)
            t += 1 / 8
            piano.notes.append(note)
        elif pitch == 128:
            if len(piano.notes) > 0:
                note = piano.notes.pop()
            else:
                p = np.random.randint(60, 72)
                note = pretty_midi.Note(
                    velocity=100, pitch=int(p), start=0, end=t)
            note = pretty_midi.Note(
                velocity=100,
                pitch=note.pitch,
                start=note.start,
                end=note.end + 1 / 8)
            piano.notes.append(note)
            t += 1 / 8
        elif pitch == 129:
            t += 1 / 8
    music.instruments.append(piano)
    music.write(output)

# if __name__ == '__main__':
#     numpy_to_midi(a)

In [13]:
def array_to_single(array):
    midi_batch = []
    for i in array:
        num = 0
        for j in i:
            if j == 1:
                midi_batch.append(num)
                num = 0
                break
            num += 1
    return midi_batch

In [14]:
# music_path = '.\output_EMOPIA\Q1__kJtgm1OUNA_0.mid'
# test = melody_to_numpy(music_path)
# # test = array_to_single(test)
# out_path = './testMidiEncode\sample.mid'
# numpy_to_midi(test,out_path)

In [15]:
import os
import pandas as pd

musicData = []
midiInfo = []
for filepath,dirname,filenames in os.walk(r'E:/Anaconda/project/envs/MusicGenPytorch/motif2MusicDemo/output_melody'):
    print("开始编码音频")
    for filename in filenames:
        try:
            musicpath = os.path.join(filepath,filename)
            midiarray = melody_to_numpy(musicpath)
            midi_batch = array_to_single(midiarray)
            lenmidi = len(midi_batch)
            midiInfo.append([lenmidi,filename])
        except Exception as e:
            print(filename+":"+str(e))
            # e

df = pd.DataFrame(midiInfo)
df.to_csv("./musicInfo.csv",index=None)

开始编码音频
ashover33.mid:index 0 is out of bounds for dimension 0 with size 0
jigs107.mid:index 0 is out of bounds for dimension 0 with size 0
jigs151.mid:index 0 is out of bounds for dimension 0 with size 0
jigs153.mid:index 0 is out of bounds for dimension 0 with size 0
jigs168.mid:index 0 is out of bounds for dimension 0 with size 0
jigs22.mid:index 0 is out of bounds for dimension 0 with size 0
jigs227.mid:index 0 is out of bounds for dimension 0 with size 0
jigs242.mid:index 0 is out of bounds for dimension 0 with size 0
jigs255.mid:index 0 is out of bounds for dimension 0 with size 0
jigs285.mid:index 0 is out of bounds for dimension 0 with size 0
jigs287.mid:index 0 is out of bounds for dimension 0 with size 0
jigs311.mid:index 0 is out of bounds for dimension 0 with size 0
jigs328.mid:index 0 is out of bounds for dimension 0 with size 0
jigs332.mid:index 0 is out of bounds for dimension 0 with size 0
jigs52.mid:index 0 is out of bounds for dimension 0 with size 0
morris10.mid:index